## 1. 必要なライブラリと関数のインポート

In [ ]:
import numpy as np
from scipy.linalg import inv, det
import warnings
from sklearn.covariance import LedoitWolf
from typing import Dict, List
import pickle

print("ライブラリをインポートしました")

## 2. 統計量計算関数の定義

In [ ]:
def estimate_single_gaussian_params(X_data: np.ndarray) -> Dict[str, np.ndarray]:
    """
    単一の高次元データセットから、多変量正規分布の平均ベクトルと共分散行列を推定する。
    サンプル数が次元数より少ない場合、Ledoit-Wolf収縮法を用いて共分散行列を頑健に推定する。

    Args:
        X_data (np.ndarray): 単一のクラスタ（またはデータセット）に属するデータ (N, D)。

    Returns:
        Dict: {'mu': 平均ベクトル, 'Sigma': 共分散行列}
    """
    N, D = X_data.shape # サンプル数 N, 次元数 D

    if N == 0:
        raise ValueError("Input data array must not be empty.")
    
    # 1. 平均ベクトル (mu) の推定
    mu = np.mean(X_data, axis=0)

    # 2. 共分散行列 (Sigma) の推定
    if N == 1:
        # サンプル数が1の場合、共分散はゼロ
        warnings.warn("N=1. Covariance matrix is set to zero (plus regularization).")
        Sigma = np.zeros((D, D))
        
    elif N < D + 1:
        # N < D+1 の場合: 特異行列になるリスクが高いため、Ledoit-Wolf収縮推定を使用
        warnings.warn(f"N={N} < D+1={D+1}. Using Ledoit-Wolf shrinkage for robust covariance estimation.")
        
        # Ledoit-Wolf収縮推定器を初期化・学習
        lw = LedoitWolf()
        lw.fit(X_data)
        Sigma = lw.covariance_
        
    else:
        # N >= D+1 の場合: 標準的な最尤推定を使用
        Sigma = np.cov(X_data, rowvar=False)

    # 3. 最終的な正則化チェック
    # 収縮推定を行ってもなお条件数が悪い場合に、微小な値を加算
    if np.linalg.cond(Sigma) > 1e15: # より厳しい条件数でチェック
        warnings.warn("Covariance matrix highly ill-conditioned. Applying final small regularization.")
        Sigma += np.eye(D) * 1e-6
        
    return {'mu': mu, 'Sigma': Sigma}


def kl_divergence_gaussian(mu1, Sigma1, mu2, Sigma2) -> float:
    """
    多変量ガウス分布 N1 から N2 へのKL情報量 D_KL(N1 || N2) を計算する。
    """
    D = mu1.shape[0]

    try:
        Sigma2_inv = inv(Sigma2)
    except np.linalg.LinAlgError:
        warnings.warn("Sigma2 is singular. KL divergence is undefined.")
        return np.nan

    # 1. ログデターミナント項
    log_det_term = np.log(det(Sigma2) / det(Sigma1))

    # 2. トレース項
    trace_term = np.trace(Sigma2_inv @ Sigma1)

    # 3. マハラノビス距離項 (平均の違い)
    diff_mu = mu2 - mu1
    mahalanobis_term = diff_mu.T @ Sigma2_inv @ diff_mu

    kl_div = 0.5 * (log_det_term + trace_term + mahalanobis_term - D)

    return kl_div


def bhattacharyya_coefficient_gaussian(mu1, Sigma1, mu2, Sigma2) -> float:
    """
    多変量ガウス分布間のバタチャリヤ係数 BC を計算する。
    値は0～1の範囲で、1に近いほど分布が重なっている。
    """
    D = mu1.shape[0]
    
    # 共分散行列の平均 Sigma = 0.5 * (Sigma1 + Sigma2)
    Sigma = 0.5 * (Sigma1 + Sigma2)

    try:
        Sigma_inv = inv(Sigma)
    except np.linalg.LinAlgError:
        warnings.warn("Sigma (mean covariance) is singular. Bhattacharyya Coefficient is undefined.")
        return np.nan

    # 1. バタチャリヤ距離の平均項（マハラノビス距離の変形）
    diff_mu = mu1 - mu2
    db_mu_term = 0.125 * diff_mu.T @ Sigma_inv @ diff_mu

    # 2. バタチャリヤ距離の共分散項
    db_cov_term = 0.5 * np.log(det(Sigma) / np.sqrt(det(Sigma1) * det(Sigma2)))

    # バタチャリヤ距離
    db_distance = db_mu_term + db_cov_term
    
    # バタチャリヤ係数 = exp(-距離)
    bc = np.exp(-db_distance)

    return bc


def mahalanobis_distance(mu1, mu2, Sigma_pooled) -> float:
    """
    プールされた共分散行列 Sigma_pooled を用いて、2つの平均間のマハラノビス距離を計算する。
    """
    try:
        Sigma_inv = inv(Sigma_pooled)
    except np.linalg.LinAlgError:
        warnings.warn("Sigma_pooled is singular. Mahalanobis distance is undefined.")
        return np.nan
    
    diff_mu = mu1 - mu2
    # 距離の2乗
    dist_sq = diff_mu.T @ Sigma_inv @ diff_mu
    
    return np.sqrt(dist_sq)


print("統計量計算関数を定義しました")

## 3. データと前処理の読み込み

以下は例です。実際の処理に合わせて変更してください。

In [ ]:
# HDBSCANクラスタリング結果の読み込み
# clusterer: HDBSCANClustererインスタンス（condensed_treeとraw_treeを含む）
# embedding: 高次元埋め込みデータ (N, D)
# node_id_map: {sequential_idx: original_cluster_id}
# id_point_map: {cluster_id: [point_indices]}

# 例：
# from hdbscan import HDBSCAN
# clusterer = HDBSCAN(min_cluster_size=...)
# clusterer.fit(X)
# embedding = X  # または他の埋め込み
# node_id_map = {...}
# id_point_map = {...}

print("データの読み込みが必要です（embeddings, node_id_map, id_point_map など）")

## 4. クラスタ間の類似度計算（メイン処理）

**動作説明：**

1. **ループ構造**：すべてのクラスタペア (i, j) で i < j となるペアのみを処理（対称性を利用して計算量を半減）

2. **クラスタ内のポイント取得**：
   - `id_point_map[cluster_id]` → そのクラスタに属するポイントインデックスのリスト
   - `embedding[point_indices]` → 当該ポイントの高次元ベクトル

3. **ガウス分布パラメータ推定**：
   - 各クラスタのポイント集合から、平均ベクトル (μ) と共分散行列 (Σ) を推定
   - Ledoit-Wolf収縮法により、サンプル不足時の特異行列化を防止

4. **類似度計算**：
   - KLダイバージェンス：分布間の情報量の差（非対称）
   - バタチャリヤ係数：分布の重なり度合い（対称、0～1）
   - マハラノビス距離：分散を考慮したクラスタ中心間の距離

5. **保存**：計算結果を辞書形式で pickle ファイルに保存

In [ ]:
# クラスタ間の類似度を事前に計算する(辞書で保存)

similarity_bc = {}
similarity_kl = {}
similarity_m = {}

cluster_ids_list = list(node_id_map.keys())
total_clusters = len(cluster_ids_list)

for i, cluster_id1 in enumerate(cluster_ids_list):
    if (i + 1) % max(1, total_clusters // 10) == 0:
        print(f"Processing cluster {i+1}/{total_clusters} (ID: {cluster_id1})")
    
    # クラスタ1のポイントとデータを取得
    point_ids1 = id_point_map[cluster_id1]
    X_data1 = embedding[point_ids1]
    params1 = estimate_single_gaussian_params(X_data1)
    
    for j, cluster_id2 in enumerate(cluster_ids_list):
        if i >= j:
            continue  # 対称性を利用して計算量削減
        
        # クラスタ2のポイントとデータを取得
        point_ids2 = id_point_map[cluster_id2]
        X_data2 = embedding[point_ids2]
        params2 = estimate_single_gaussian_params(X_data2)
        
        # KLダイバージェンス
        kl_div = kl_divergence_gaussian(params1['mu'], params1['Sigma'],
                                        params2['mu'], params2['Sigma'])
        
        # バタチャリヤ係数
        bc = bhattacharyya_coefficient_gaussian(params1['mu'], params1['Sigma'],
                                                params2['mu'], params2['Sigma'])
        
        # マハラノビス距離
        pooled_cov = 0.5 * (params1['Sigma'] + params2['Sigma'])
        m_dist = mahalanobis_distance(params1['mu'], params2['mu'], pooled_cov)
        
        # 辞書に保存（cluster_id1, cluster_id2）をキーとして
        similarity_kl[(cluster_id1, cluster_id2)] = kl_div
        similarity_bc[(cluster_id1, cluster_id2)] = bc
        similarity_m[(cluster_id1, cluster_id2)] = m_dist

print(f"\n計算完了：{len(similarity_kl)} クラスタペア")

## 5. 結果の確認

In [ ]:
# 結果の統計情報を表示
kl_values = list(similarity_kl.values())
bc_values = list(similarity_bc.values())
m_values = list(similarity_m.values())

print("KL Divergence 統計:")
print(f"  Min: {np.nanmin(kl_values):.4f}")
print(f"  Max: {np.nanmax(kl_values):.4f}")
print(f"  Mean: {np.nanmean(kl_values):.4f}")
print(f"  NaN count: {np.sum(np.isnan(kl_values))}")

print("\nBhattacharyya Coefficient 統計:")
print(f"  Min: {np.nanmin(bc_values):.4f}")
print(f"  Max: {np.nanmax(bc_values):.4f}")
print(f"  Mean: {np.nanmean(bc_values):.4f}")
print(f"  NaN count: {np.sum(np.isnan(bc_values))}")

print("\nMahalanobis Distance 統計:")
print(f"  Min: {np.nanmin(m_values):.4f}")
print(f"  Max: {np.nanmax(m_values):.4f}")
print(f"  Mean: {np.nanmean(m_values):.4f}")
print(f"  NaN count: {np.sum(np.isnan(m_values))}")

## 6. 結果の保存

In [ ]:
# save as file
with open("cluster_similarities.pkl", "wb") as f:
    pickle.dump({
        "kl_divergence": similarity_kl,
        "bhattacharyya_coefficient": similarity_bc,
        "mahalanobis_distance": similarity_m
    }, f)

print("結果をファイルに保存しました: cluster_similarities.pkl")